# Загрузка документов и сохранение эмбеддингов в базу

In [1]:
import yaml
import arxiv
import os
from langchain_community.document_loaders import PyPDFLoader
import chromadb
from tqdm import tqdm
import logging

In [2]:
# === Считывание тем из YAML ===
def load_topics_from_yaml(file_path: str):
    """Считывает YAML файл и возвращает список тем."""
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data['topics']

In [3]:
# === Загрузка статей с arXiv ===
def download_arxiv_papers(query: str, max_results: int, download_folder: str):
    """Загружает статьи с arXiv по запросу."""
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    search = arxiv.Search(
        query=query,
        max_results=max_results,
        # sort_by=arxiv.SortCriterion.SubmittedDate
    )

    for result in search.results():
        paper_id = result.entry_id.split('/')[-1]
        pdf_path = os.path.join(download_folder, f'{paper_id}.pdf')
        if not os.path.exists(pdf_path):
            print(f"Downloading {result.title}...")
            result.download_pdf(download_folder, f'{paper_id}.pdf')
            print(f"Saved to {pdf_path}")
        else:
            print(f"{result.title} already downloaded.")

In [4]:
# === Считывание текста из PDF ===
def extract_text_from_pdf(file_path: str, extract_images=True):
    """Извлекает текст из PDF файла."""
    loader = PyPDFLoader(file_path, extract_images=extract_images)
    pages = loader.load()
    document_text = ' '.join([page.page_content for page in pages])
    return document_text, pages

In [5]:
# === Загрузка данных в ChromaDB ===
def upload_to_chromadb(documents: list, collection_name: str, db_path: str = "./chroma_storage"):
    """Добавляет документы в коллекцию ChromaDB."""
    client = chromadb.PersistentClient(path=db_path)
    collection = client.get_or_create_collection(name=collection_name)

    for doc in documents:
        collection.add(
            ids=[doc["ids"]],
            documents=[doc["content"]],
            metadatas=[doc["metadata"]]
        )
    print(f"Uploaded {len(documents)} documents to collection '{collection_name}'.")

In [6]:
# Настройка логирования
logging.basicConfig(
    filename="log/error_arxiv.txt",
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s"
)


# Запуск

In [7]:
# Шаг 1. Считываем темы из YAML
topics = load_topics_from_yaml('config/topics.yaml')
print(f"Loaded topics: {[topic['name'] for topic in topics]}")

Loaded topics: ['Machine Learning', 'Data Analysis', 'Optimization Techniques', 'Natural Language Processing', 'Computer Vision', 'Theoretical Foundations', 'Applied AI']


In [ ]:
# Шаг 2. Загрузка статей по каждой теме

for topic in tqdm(topics):
    topic_name = topic['name']
    folder = f"dataset/{topic_name}"#.replace(' ', '_')}"

    for keyword in topic['keywords']:
        query = f"all:\"{keyword}\""
        try:
            download_arxiv_papers(query, max_results=20, download_folder=folder + f'/{keyword}')
        except Exception as e:
            # Логирование ошибки
            logging.error(f"Ошибка при загрузке статей для темы '{topic_name}', ключевого слова '{keyword}': {e}")
            # (Необязательно) Вывод сообщения об ошибке в консоль
            print(f"Ошибка: {e} - при обработке темы '{topic_name}', ключевого слова '{keyword}'")

  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\Igorexy\AppData\Local\Temp\ipykernel_19820\3574635866.py:13: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Saved to dataset/Theoretical Foundations/probability theory\9509004v1.pdf
Saved to dataset/Theoretical Foundations/probability theory\1909.07375v3.pdf
Saved to dataset/Theoretical Foundations/probability theory\0706.0750v2.pdf
Saved to dataset/Theoretical Foundations/probability theory\1602.08867v1.pdf
Saved to dataset/Theoretical Foundations/probability theory\2006.01607v1.pdf
Saved to dataset/Theoretical Foundations/probability theory\2012.06355v1.pdf
Saved to dataset/Theoretical Foundations/probability theory\0110253v2.pdf
Saved to dataset/Theoretical Foundations/probability theory\1608.00141v1.pdf
Saved to dataset/Theoretical Foundations/probability theory\0304341v1.pdf
Saved to dataset/Theoretical Foundations/probability theory\1905.03409v1.pdf
Saved to dataset/Theoretical Foundations/probability theory\2004.03861v2.pdf
Saved to dataset/Theoretical Foundations/probability theory\1302.3684v3.pdf
Saved to dataset/Theoretical Foundations/probability theory\1302.5325v4.pdf
Saved to da

 50%|█████     | 1/2 [01:34<01:34, 94.62s/it]

Saved to dataset/Theoretical Foundations/Bayesian inference\1606.07937v1.pdf
Saved to dataset/Applied AI/recommender systems\2411.07589v1.pdf
Saved to dataset/Applied AI/recommender systems\2208.09864v1.pdf
Saved to dataset/Applied AI/recommender systems\2208.11384v1.pdf
Saved to dataset/Applied AI/recommender systems\2212.06750v1.pdf
Saved to dataset/Applied AI/recommender systems\2404.17844v1.pdf
Saved to dataset/Applied AI/recommender systems\1511.05263v4.pdf
Saved to dataset/Applied AI/recommender systems\1805.06594v1.pdf
Saved to dataset/Applied AI/recommender systems\2004.03774v2.pdf
Saved to dataset/Applied AI/recommender systems\2302.09803v1.pdf
Saved to dataset/Applied AI/recommender systems\2306.12680v1.pdf
Saved to dataset/Applied AI/recommender systems\2409.15060v1.pdf
Saved to dataset/Applied AI/recommender systems\2107.06590v1.pdf
Saved to dataset/Applied AI/recommender systems\2008.07759v1.pdf
Saved to dataset/Applied AI/recommender systems\2212.13910v1.pdf
Saved to data

100%|██████████| 2/2 [05:09<00:00, 154.71s/it]

Saved to dataset/Applied AI/AI ethics\2206.07635v2.pdf


In [ ]:
# Шаг 3. Извлечение текста из PDF

all_documents = []
for topic in [topics[0]]:
    topic_name = topic['name']
    folder = rf"dataset\{topic_name}"
    for keyword in os.listdir(folder):
        folder_keywords = os.path.join(folder, keyword)
        print(folder_keywords)
        for file_name in os.listdir(folder_keywords):
            if file_name.endswith('.pdf'):
                file_path = os.path.join(folder_keywords, file_name)
                try:
                    # document_text, pages = extract_text_from_pdf(file_path, extract_images=False)
                    document_text, pages = extract_text_from_pdf(file_path, extract_images=False)
                except Exception as e:
                    logging.error(f"Ошибка при считывани текста для темы '{topic_name}', ключевого слова '{keyword}': {e}")
                    print(f"Ошибка: {e} - при обработке темы '{topic_name}', ключевого слова '{keyword}'")
                all_documents.append({
                    "ids": file_name.split('.pdf')[0],
                    "content": document_text,
                    "metadata": {
                        "topic": topic_name,
                        "keyword": keyword,  # Добавляем ключевое слово в метаданные
                        "filename": file_name
                    }
                })
print(f"Extracted text from {len(all_documents)} documents.")

#### Модифицированное извлечение текста

In [7]:
# Шаг 1. Считываем темы из YAML
topics = load_topics_from_yaml('config/topics.yaml')
print(f"Loaded topics: {[topic['name'] for topic in topics]}")

Loaded topics: ['Machine Learning', 'Data Analysis', 'Optimization Techniques', 'Natural Language Processing', 'Computer Vision', 'Theoretical Foundations', 'Applied AI']


In [9]:
# === Считывание текста из PDF ===
import re

def mode_extract_text_from_pdf(file_path: str, extract_images=True):
    """Извлекает текст из PDF файла."""
    loader = PyPDFLoader(file_path, extract_images=extract_images)
    pages = loader.load()

    for i in range(len(pages)):
        cleaned_page = pages[i].page_content

        # Удаляем спецсимволы, оставляя буквы, цифры, пробелы и пунктуацию
        cleaned_page = re.sub(r'[^а-яА-ЯёЁa-zA-Z0-9\s.,!?-]', '', cleaned_page)

        # Убираем лишние переносы строк
        cleaned_page = re.sub(r'\n{2,}', '\n', cleaned_page)

        # Удаляем ссылки на литературу вида [1], [2], ...
        cleaned_page = re.sub(r'\[\d+\]', '', cleaned_page)

        # Удаляем email-адреса
        cleaned_page = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', cleaned_page)

        # Удаляем артикулы на arXiv
        cleaned_page = re.sub(r'arXiv:\d+\.\d+(v\d+)? \[\w+(\.\w+)?\] \d+ \w+ \d+', '', cleaned_page)

        # Удаляем формулы, заключенные в $...$ или \( ... \)
        cleaned_page = re.sub(r'\$.*?\$|\\\(.*?\\\)', '', cleaned_page)

        # Удаляем числа
        cleaned_page = re.sub(r'\d+', '', cleaned_page)

        # Удаляем математические символы и спецсимволы
        cleaned_page = re.sub(r'[+\-*/=<>^~_|&%$#@!]', '', cleaned_page)

        # Удаляем гиперссылки
        cleaned_page = re.sub(r'http\S+', '', cleaned_page)

        # Убираем лишние пробелы
        cleaned_page = re.sub(r'\s+', ' ', cleaned_page).strip()

        # Проверяем наличие слова REFERENCES (регистр не учитывается)
        if re.search(r'\bREFERENCES\b', cleaned_page, flags=re.IGNORECASE):
            # Удаляем текст после "REFERENCES" (регистр не имеет значения)
            cleaned_page = re.split(r'\bREFERENCES\b', cleaned_page, flags=re.IGNORECASE)[0]
            pages[i].page_content = cleaned_page
            break

        # Сохраняем очищенный текст обратно в страницу
        pages[i].page_content = cleaned_page


    # Объединяем текст всех страниц в один документ
    document_text = ' '.join([page.page_content for page in pages])

    return document_text, pages


In [12]:
# Шаг 3. Извлечение текста из PDF

all_documents = []

# for topic in [topics[0]]:
for topic in topics:
    topic_name = topic['name']
    folder = rf"dataset\{topic_name}"

    for keyword in os.listdir(folder):
        folder_keywords = os.path.join(folder, keyword)
        print(folder_keywords)

        for file_name in os.listdir(folder_keywords):
            if file_name.endswith('.pdf'):
                file_path = os.path.join(folder_keywords, file_name)

                try:
                    # Предполагается, что функция extract_text_from_pdf возвращает общий текст и список страниц
                    document_text, pages = mode_extract_text_from_pdf(file_path, extract_images=False)
                except Exception as e:
                    logging.error(f"Ошибка при считывании текста для темы '{topic_name}', ключевого слова '{keyword}': {e}")
                    print(f"Ошибка: {e} - при обработке темы '{topic_name}', ключевого слова '{keyword}'")
                    continue

                # Формируем записи для каждой страницы
                for i, page in enumerate(pages):
                    all_documents.append({
                        "ids": file_name.split('.pdf')[0] + f" {str(i)}",
                        "content": page.page_content,
                        "metadata": {
                            "topic": topic_name,
                            "keyword": keyword,  # Добавляем ключевое слово в метаданные
                            "filename": file_name,
                            "page_count": len(pages)  # Число страниц в документе
                        }
                    })

print(f"Extracted text from {len(all_documents)} documents.")

dataset\Machine Learning\clustering


KeyboardInterrupt: 

В прошлый раз ушло 2 минуты на считывание только Machine Learning

In [11]:
# Шаг 4. Загрузка документов в ChromaDB
upload_to_chromadb(all_documents, collection_name="magic_document")

Uploaded 16618 documents to collection 'magic_document'.
